In [222]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy.sparse.linalg import svds
import math

from recsys.preprocess import *
from recsys.utility import *

import functools

RANDOM_STATE = 2342

np.random.seed(RANDOM_STATE)

%matplotlib inline

In [2]:
train = pd.read_csv('data/train_final.csv', delimiter='\t')
playlists = pd.read_csv('data/playlists_final.csv', delimiter='\t')
target_playlists = pd.read_csv('data/target_playlists.csv', delimiter='\t')
target_tracks = pd.read_csv('data/target_tracks.csv', delimiter = '\t')
tracks = pd.read_csv('data/tracks_final.csv', delimiter='\t')

In [ ]:
def reduce_train(train, to_keep=0.8):
    # shuffle train index
    train.drop(range(int(len(train)*to_keep))).head()
    
#train = reduce_train(train, to_keep=0.5)

In [3]:
train, test, target_playlists, target_tracks = train_test_split(train, test_size=0.30, min_playlist_tracks=10)

# Preprocessing

In [4]:
tracks['track_id_tmp'] = tracks['track_id']

tracks['track_id'] = tracks.index

playlists['playlist_id_tmp'] = playlists['playlist_id']
playlists['playlist_id'] = playlists.index

train['playlist_id_tmp'] = train['playlist_id']
train['track_id_tmp'] = train['track_id']

track_to_num = pd.Series(tracks.index)
track_to_num.index = tracks['track_id_tmp']

playlist_to_num = pd.Series(playlists.index)
playlist_to_num.index = playlists['playlist_id_tmp']

num_to_tracks = pd.Series(tracks['track_id_tmp'])


train['track_id'] = train['track_id'].apply(lambda x : track_to_num[x])
train['playlist_id'] = train['playlist_id'].apply(lambda x : playlist_to_num[x])

In [5]:
tracks.tags = tracks.tags.apply(lambda s: np.array(eval(s), dtype=int))

In [32]:
playlists.title = playlists.title.apply(lambda s: np.array(eval(s), dtype=int))

In [6]:
tracks.loc[0].tags

array([ 54087,   1757,   1718, 116712, 189631])

In [8]:
playlists.head()

,created_at,playlist_id,title,numtracks,duration,owner,playlist_id_tmp
0,1216545588,0,[12727],27,6522,41504,644838
1,1249326867,1,[],9,2650,41504,7577564
2,1257766688,2,[183],16,3645,44542,3120683
3,1248079275,3,"[12389, 18698, 18925, 11695, 7117]",15,4151,44542,4278112
4,1175201268,4,"[12809, 2095, 13257, 12671, 20426, 14448, 18698]",84,18414,44542,8656823


In [9]:
train.head()

,playlist_id,track_id,playlist_id_tmp,track_id_tmp
0,5501,65131,3271849,2801526
1,46691,90762,5616275,727878
3,402,37363,10103900,1515105
4,28466,84016,3836898,2945623
5,23632,26065,5270369,2821391


In [10]:
track_to_num.head()

track_id_tmp
2972914    0
2750239    1
1550729    2
2169950    3
1903709    4
dtype: int64

In [11]:
playlist_to_num[:5]

playlist_id_tmp
644838     0
7577564    1
3120683    2
4278112    3
8656823    4
dtype: int64

In [12]:
num_to_tracks[:5]

0    2972914
1    2750239
2    1550729
3    2169950
4    1903709
Name: track_id_tmp, dtype: int64

In [13]:
target_playlists['playlist_id_tmp'] = target_playlists['playlist_id']
target_playlists['playlist_id'] = target_playlists['playlist_id'].apply(lambda x : playlist_to_num[x])

target_tracks['track_id_tmp'] = target_tracks['track_id']
target_tracks['track_id'] = target_tracks['track_id'].apply(lambda x : track_to_num[x])

In [14]:
target_tracks.head()

,track_id,track_id_tmp
0,44292,527.0
1,81953,667.0
2,11782,874.0
3,61792,1450.0
4,20686,2352.0


In [15]:
target_playlists.head()

,playlist_id,playlist_id_tmp
0,27008,8783912
1,15351,10755431
2,39931,4366846
3,57170,5766032
4,36487,5253992


In [16]:
playlist_tracks = pd.DataFrame(train['playlist_id'].drop_duplicates())
playlist_tracks.index = train['playlist_id'].unique()
playlist_tracks['track_ids'] = train.groupby('playlist_id').apply(lambda x : x['track_id'].values)
playlist_tracks = playlist_tracks.sort_values('playlist_id')

In [17]:
playlist_tracks.head()

,playlist_id,track_ids
0,0,"[45626, 45865, 63716, 82189, 85085, 29862, 881..."
1,1,"[76252, 34102, 71429, 71433, 63708, 20189]"
2,2,"[78331, 77629, 76454, 47409, 58892, 75023, 38754]"
3,3,"[58886, 67979, 61169, 15486, 48141, 73222, 632..."
4,4,"[51958, 66515, 99106, 33273, 9083, 41552, 9020..."


In [18]:
track_playlists = pd.DataFrame(train['track_id'].drop_duplicates())
track_playlists.index = train['track_id'].unique()
track_playlists['playlist_ids'] = train.groupby('track_id').apply(lambda x : x['playlist_id'].values)
track_playlists = track_playlists.sort_values('track_id')

In [19]:
track_playlists.head()

,track_id,playlist_ids
0,0,"[40622, 6896, 23639, 22163, 13179, 34564, 4855..."
1,1,"[53383, 53366, 55805, 25094, 45977, 53378, 961..."
2,2,"[11101, 40372, 49479, 27385, 40621, 54319, 546..."
3,3,"[44104, 53823, 50424, 39587, 54319, 6896, 2216..."
4,4,"[50423, 22163, 40622, 18519]"


In [20]:
def transform_album_1(alb):
    ar = eval(alb)
    if len(ar) == 0 or (len(ar) > 0 and ar[0] == None):
        ar = [-1]
    return ar[0]

def transform_album_2(alb):
    global next_album_id
    if alb == -1:
        alb = next_album_id
        next_album_id += 1
    return alb
    
tracks.album = tracks.album.apply(lambda alb: transform_album_1(alb))

last_album = tracks.album.max()
next_album_id = last_album + 1

tracks.album = tracks.album.apply(lambda alb: transform_album_2(alb))

# Clean data

### URM and SVD decomposition

In [21]:
# User Rating Matrix URM
def get_URM(tracks, playlists, playlist_tracks, track_playlists, normalized=False):
    URM = lil_matrix((len(playlists), len(tracks)))
    num_playlists = len(playlist_tracks)

    i = 0
    
    for row in track_playlists.itertuples():
        track_id = row.track_id
        #row.playlist_ids.sort()
        nq = len(row.playlist_ids)
        for pl_id in row.playlist_ids:
            URM[pl_id,track_id] = math.log((num_playlists - nq + 0.5)/(nq + 0.5)) if normalized else 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return URM

In [22]:
%%time
URM = get_URM(tracks, playlists, playlist_tracks, track_playlists, normalized=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
CPU times: user 12.7 s, sys: 212 ms, total: 12.9 s
Wall time: 13 s


In [115]:
URM = URM.tocsc()

In [116]:
%%time
U, S, V = svds(URM, k=200)

CPU times: user 2min, sys: 2.49 s, total: 2min 3s
Wall time: 1min 7s


In [117]:
S = np.diag(S)

In [118]:
M2 = np.dot(S, V)



Normalized:<br>
k = 50 -> 0.012786324786324807<br>
k = 200 -> 0.024994972347913386<br>
k = 500 -> 0.0353849999999999

### Tags

In [100]:
# Count distinct tags
tag_tracks = {}
for row in tracks.itertuples():
    for tag in row.tags:
        if tag in tag_tracks:
            tag_tracks[tag].append(row.track_id)
        else:
            tag_tracks[tag] = [row.track_id]

In [101]:
# Item Tag Matrix ITM
def get_ITM(tracks, tag_tracks, normalized=False):
    unique_tags = list(tag_tracks.keys())
    ITM = lil_matrix((len(tracks), max(unique_tags)+1))
    ITM_count = lil_matrix((len(tracks), max(unique_tags)+1))
    
    num_tracks = len(tracks)
    i = 0
    
    for tag,track_ids in tag_tracks.items():
        #row.playlist_ids.sort()
        nq = len(track_ids)
        for track_id in track_ids:
            ITM[track_id,tag] = math.log((num_tracks - nq + 0.5)/(nq + 0.5)) if normalized else 1
            ITM_count[track_id,tag] = 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return ITM

In [102]:
ITM = get_ITM(tracks, tag_tracks, normalized=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [103]:
"""def create_row(row_num, tags_concatenated):
    tags_concatenated.sort()
    d = np.array([])
    r = np.array([])
    c = np.array([])
    for i,tag in enumerate(tags_concatenated):
        if i > 0 and tags_concatenated[i-1] == tags_concatenated[i]:
            d[-1] += 1
        else:
            d = np.append(d,1)
            r = np.append(r,row_num)
            c = np.append(c,tags_concatenated[i])
    return d, (r, c)
"""

# User Tag Matrix UTM
def get_UTM(tracks, playlist_tracks, tag_tracks, OKAPI_K=1.7, OKAPI_B=0.75):
    unique_tags = list(tag_tracks.keys())
    
    i = 0
    """
    d = np.array([])
    r = np.array([])
    c = np.array([])
    for row in playlist_tracks.itertuples():
        pl_id = row.playlist_id
        tags_concatenated = np.array([])
        for tr_id in row.track_ids:
            tags = tracks.loc[tr_id].tags
            tags_concatenated = np.concatenate((tags_concatenated, tags))
        d1, (r1, c1) = create_row(row.playlist_id, tags_concatenated)
        
        d = np.concatenate((d, d1))
        r = np.concatenate((r, r1))
        c = np.concatenate((c, c1))
        
        i += 1
        if i % 1000 == 0:
            print(i)
    UTM = coo_matrix(d, (r, c))
    """
    UTM = lil_matrix((max(playlists.playlist_id)+1, max(unique_tags)+1))
    for row in playlist_tracks.itertuples():
        pl_id = row.playlist_id
        for tr_id in row.track_ids:
            for tag in tracks.loc[tr_id].tags:
                UTM[pl_id,tag] += 1
                
        i += 1
        if i % 1000 == 0:
            print(i)
    
    avg_document_length = sum(list(map(lambda l: sum(l), UTM.data)))/len(UTM.data)
    
    i = 0
    
    for row in playlist_tracks.itertuples():
        pl_id = row.playlist_id
        tags = UTM.rows[pl_id]
        data = UTM.data[pl_id]
        for tag in tags:
            fq = UTM[pl_id,tag]
            UTM[pl_id,tag] = (fq*(OKAPI_K+1))/(fq + OKAPI_K*(1 - OKAPI_B + OKAPI_B * sum(data) / avg_document_length))
        i += 1
        if i % 1000 == 0:
            print(i)
    
    return UTM

In [104]:
UTM = get_UTM(tracks, playlist_tracks, tag_tracks)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [105]:
UTM_csc = UTM.tocsc()

In [106]:
ITM_csr_transpose = ITM.tocsr().transpose()

### Artists

In [213]:
unique_artists = tracks.artist_id.unique()

In [216]:
# Item Artist Matrix
def get_IAM(tracks, target_tracks, normalized=False):
    unique_artists = tracks.artist_id.unique()
    IAM = lil_matrix((len(tracks), max(unique_artists)+1))
    tracks_filtered = tracks[tracks.track_id.isin(target_tracks.track_id)]
    
    num_tracks = len(tracks)
    i = 0
    
    for row in tracks_filtered.itertuples():
        nq = 1
        IAM[row.track_id,row.artist_id] = math.log((num_tracks - nq + 0.5)/(nq + 0.5)) if normalized else 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return IAM

In [217]:
IAM = get_IAM(tracks, target_tracks, normalized=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [228]:
# User Artist Matrix UAM
def get_UAM(tracks, playlist_tracks, target_playlists, OKAPI_K=1.7, OKAPI_B=0.75):
    unique_artists = tracks.artist_id.unique()
    playlist_tracks_filtered = playlist_tracks[playlist_tracks.playlist_id.isin(target_playlists.playlist_id)]
    
    i = 0

    UAM = lil_matrix((max(playlists.playlist_id)+1, max(unique_artists)+1))
    for row in playlist_tracks_filtered.itertuples():
        pl_id = row.playlist_id
        for tr_id in row.track_ids:
            UAM[pl_id,tracks.loc[tr_id].artist_id] += 1
                
        i += 1
        if i % 1000 == 0:
            print(i)
    
    avg_document_length = functools.reduce(lambda acc,tr_ids: acc + len(tr_ids), playlist_tracks.track_ids, 0) / len(playlist_tracks)
    #avg_document_length = sum(list(map(lambda l: sum(l), UAM.data)))/len(UAM.data)
    
    i = 0
    
    for row in playlist_tracks_filtered.itertuples():
        pl_id = row.playlist_id
        artists = UAM.rows[pl_id]
        data = UAM.data[pl_id]
        for artist in artists:
            fq = UAM[pl_id,artist]
            UAM[pl_id,artist] = (fq*(OKAPI_K+1))/(fq + OKAPI_K*(1 - OKAPI_B + OKAPI_B * sum(data) / avg_document_length))
        i += 1
        if i % 1000 == 0:
            print(i)
    
    return UAM

In [229]:
UAM = get_UAM(tracks, playlist_tracks, target_playlists, OKAPI_K=1.7, OKAPI_B=0.75)

1000
2000
3000
4000
5000
6000
1000
2000
3000
4000
5000
6000


In [230]:
UAM_csc = UAM.tocsc()
IAM_csr_transpose = IAM.tocsr().transpose()

### Albums

In [342]:
unique_albums = tracks.album.unique()
unique_albums

array([     7,      8,      9, ..., 244079, 244080, 244081])

In [343]:
# Item Album Matrix IAM_album
def get_IAM_album(tracks, target_tracks, normalized=False):
    unique_albums = tracks.album.unique()
    IAM_album = lil_matrix((len(tracks), max(unique_albums)+1))
    tracks_filtered = tracks[tracks.track_id.isin(target_tracks.track_id)]
    
    num_tracks = len(tracks)
    i = 0
    
    for row in tracks_filtered.itertuples():
        nq = 1
        IAM_album[row.track_id,row.album] = math.log((num_tracks - nq + 0.5)/(nq + 0.5)) if normalized else 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return IAM_album

In [345]:
IAM_album = get_IAM_album(tracks, target_tracks, normalized=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [346]:
# User Album Matrix UAM_album
def get_UAM_album(tracks, playlist_tracks, target_playlists, OKAPI_K=1.7, OKAPI_B=0.75):
    unique_albums = tracks.album.unique()
    playlist_tracks_filtered = playlist_tracks[playlist_tracks.playlist_id.isin(target_playlists.playlist_id)]
    
    i = 0

    UAM_album = lil_matrix((max(playlists.playlist_id)+1, max(unique_albums)+1))
    for row in playlist_tracks_filtered.itertuples():
        pl_id = row.playlist_id
        for tr_id in row.track_ids:
            UAM_album[pl_id,tracks.loc[tr_id].album] += 1
                
        i += 1
        if i % 1000 == 0:
            print(i)
    
    avg_document_length = functools.reduce(lambda acc,tr_ids: acc + len(tr_ids), playlist_tracks.track_ids, 0) / len(playlist_tracks)
    #avg_document_length = sum(list(map(lambda l: sum(l), UAM_album.data)))/len(UAM_album.data)
    
    i = 0
    
    for row in playlist_tracks_filtered.itertuples():
        pl_id = row.playlist_id
        albums = UAM_album.rows[pl_id]
        data = UAM_album.data[pl_id]
        for album in albums:
            fq = UAM_album[pl_id,album]
            UAM_album[pl_id,album] = (fq*(OKAPI_K+1))/(fq + OKAPI_K*(1 - OKAPI_B + OKAPI_B * sum(data) / avg_document_length))
        i += 1
        if i % 1000 == 0:
            print(i)
    
    return UAM_album

In [348]:
UAM_album = get_UAM_album(tracks, playlist_tracks, target_playlists, OKAPI_K=1.7, OKAPI_B=0.75)

1000
2000
3000
4000
5000
6000
1000
2000
3000
4000
5000
6000


In [349]:
UAM_album_csc = UAM_album.tocsc()
IAM_album_csr_transpose = IAM_album.tocsr().transpose()

### Playlist titles

In [238]:
def from_num_to_id(df, row_num, column = 'track_id'):
    """ df must have a 'track_id' column """
    return df.iloc[row_num][column]

def from_id_to_num(df, tr_id, column='track_id'):
    """ df must have a 'track_id' column """
    return np.where(df[column].values == tr_id)[0][0]

In [241]:
# Count distinct title tokens
token_playlists = {}
for row in playlists.itertuples():
    for token in row.title:
        if token in token_playlists:
            token_playlists[token].append(row.playlist_id)
        else:
            token_playlists[token] = [row.playlist_id]
            
token_playlists_filtered = {}
for row in playlists[playlists.playlist_id.isin(target_playlists.playlist_id)].itertuples():
    for token in row.title:
        if token in token_playlists_filtered:
            token_playlists_filtered[token].append(row.playlist_id)
        else:
            token_playlists_filtered[token] = [row.playlist_id]

In [247]:
# User Title Matrix UTM_title
def get_UTM_title(playlists, target_playlists, token_playlists, token_playlists_filtered, normalized=False):
    unique_tokens = list(token_playlists.keys())
    UTM_title = lil_matrix((len(target_playlists), max(unique_tokens)+1))
    playlists_filtered = playlists[playlists.playlist_id.isin(target_playlists.playlist_id)]
    
    num_playlists = len(playlists)
    i = 0
    
    for token,playlist_ids in token_playlists_filtered.items():
        nq = len(token_playlists[token])
        for playlist_id in playlist_ids:
            UTM_title[from_id_to_num(target_playlists, playlist_id, column='playlist_id'),token] = math.log((num_playlists - nq + 0.5)/(nq + 0.5)) if normalized else 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return UTM_title

In [326]:
UTM_title = get_UTM_title(playlists, target_playlists, token_playlists, token_playlists_filtered, normalized=True)

0
1000
2000
3000
4000


In [327]:
UTM_title

<6630x21064 sparse matrix of type '<class 'numpy.float64'>'
	with 13543 stored elements in LInked List format>

In [328]:
UTM_title = UTM_title.tocsr()
UTM_title_transpose = UTM_title.transpose().tocsc()

In [329]:
PS_title = np.dot(UTM_title, UTM_title_transpose)

In [330]:
PS_title = PS_title.todense()

In [331]:
PS_title.max()

894.26244503975374

In [263]:
# User Rating Matrix URM
def get_URM_target(tracks, playlists, playlist_tracks, track_playlists, target_playlists, target_tracks, normalized=False):
    URM = lil_matrix((len(target_playlists), len(target_tracks)))
    num_playlists = len(playlist_tracks)

    i = 0
    
    for row in track_playlists[track_playlists.track_id.isin(target_tracks.track_id)].itertuples():
        track_id = row.track_id
        #row.playlist_ids.sort()
        nq = len(row.playlist_ids)
        for pl_id in row.playlist_ids:
            if pl_id in target_playlists.playlist_id:
                URM[from_id_to_num(target_playlists, pl_id, column='playlist_id'),from_id_to_num(target_tracks, track_id, column='track_id')] = math.log((num_playlists - nq + 0.5)/(nq + 0.5)) if normalized else 1
        if i % 1000 == 0:
            print(i)
        i += 1
    
    return URM

In [332]:
URM_target = get_URM_target(tracks, playlists, playlist_tracks, track_playlists, target_playlists, target_tracks, normalized=False)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [333]:
URM_target

<6630x23598 sparse matrix of type '<class 'numpy.float64'>'
	with 113060 stored elements in LInked List format>

In [334]:
URM_target = URM_target.tocsc()

In [335]:
PS_title_csr = csr_matrix(PS_title)
PS_title_csr

<6630x6630 sparse matrix of type '<class 'numpy.float64'>'
	with 835329 stored elements in Compressed Sparse Row format>

In [336]:
np.array(np.dot(PS_title_csr[0], URM_target).todense())[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

### Duration 

In [149]:
def get_avg_duration(tr_ids):
    sum_durations = 0
    tracks_to_count = 0
    for tr_id in tr_ids:
        tr_dur = tracks.loc[tr_id].duration
        if tr_dur >= 0:
            sum_durations += tr_dur
            tracks_to_count += 1
    return 0 if tracks_to_count == 0 else sum_durations/tracks_to_count

MAX_DURATION = 400
tracks.duration = tracks.duration.apply(lambda dur: dur/1000) # to seconds
tracks.duration = tracks.duration.apply(lambda dur: min(dur, MAX_DURATION)) # clamp
playlist_tracks["avg_duration"] = playlist_tracks.track_ids.apply(lambda tr_ids: get_avg_duration(tr_ids))

In [162]:
DURATION_K = 400

1.4709293982627514

### Playcount

In [182]:
def get_avg_playcount(tr_ids):
    sum_playcounts = 0
    tracks_to_count = 0
    for tr_id in tr_ids:
        tr_plc = tracks.loc[tr_id].playcount
        if tr_plc >= 0:
            sum_playcounts += tr_plc
            tracks_to_count += 1
    return 0 if tracks_to_count == 0 else sum_playcounts/tracks_to_count

playlist_tracks["avg_playcount"] = playlist_tracks.track_ids.apply(lambda tr_ids: get_avg_playcount(tr_ids))

In [185]:
playlist_tracks.avg_playcount[:20]

0      3774.950000
1       882.500000
2       270.714286
3      1627.333333
4      4164.766667
5      3261.466667
6      1783.323529
7      1352.000000
8      1069.000000
9     26874.270270
11       26.500000
12     1305.666667
13      179.800000
14     1027.444444
15     2051.500000
16     4117.833333
17     3762.460870
23    13246.000000
24     1546.888889
25     6136.818182
Name: avg_playcount, dtype: float64

In [183]:
PLAYCOUNT_K = 2500

### Predictions

In [364]:
ALPHA = 1
BETA = 0.9
GAMMA = 0.5

In [365]:
def make_predictions(test=None, compute_MAP=False):
    predictions = pd.DataFrame(target_playlists)
    predictions.index = target_playlists['playlist_id']
    predictions['track_ids'] = [np.array([]) for i in range(len(predictions))]
    ttracks = set(target_tracks['track_id'].values)
    test_good = get_playlist_track_list2(test)
    test_good.index = test_good.playlist_id.apply(lambda pl_id: playlist_to_num[pl_id])
    
    counter = 0
    mean_ap = 0
    
    for _,row in target_playlists.iterrows():
        # Compute predictions for current playlist
        pred = []
        pl_id = row['playlist_id']
        pl_tracks = set(playlist_tracks.loc[pl_id]['track_ids'])
        simil = ALPHA * np.array(np.dot(UAM_album_csc[pl_id,:], IAM_album_csr_transpose).todense())[0]
        simil += BETA * np.array(np.dot(UAM_csc[pl_id,:], IAM_csr_transpose).todense())[0]
        #simil = np.array(np.dot(UAM_csc[pl_id,:], IAM_csr_transpose).todense())[0]
        #simil = np.array(np.dot(PS_title_csr[from_id_to_num(target_playlists, pl_id, column='playlist_id'),:], URM_target).todense())[0]
        simil += GAMMA * np.array(np.dot(UTM_csc[pl_id,:], ITM_csr_transpose).todense())[0]
        #simil += DELTA * np.dot(U[pl_id,:], M2)
        #simil = np.exp(-(np.abs(playlist_tracks.loc[pl_id].avg_duration - tracks.duration))/DURATION_K)
        #simil = np.exp(-(np.abs(playlist_tracks.loc[pl_id].avg_playcount - tracks.playcount))/PLAYCOUNT_K).dropna()
        sorted_ind = simil.argsort()
        i = len(sorted_ind) - 1
        c = 0
        while i > 0 and c < 5:
            #tr = from_num_to_id(target_tracks, sorted_ind[i], column='track_id')
            tr = sorted_ind[i]
            if (tr in ttracks) and (tr not in pl_tracks):
                pred.append(num_to_tracks[tr])
                c+=1
            i-=1
        predictions.loc[row['playlist_id']] = predictions.loc[row['playlist_id']].set_value('track_ids', np.array(pred))

        # Update MAP
        if compute_MAP:
            correct = 0
            ap = 0
            for it, t in enumerate(pred):
                if t in test_good.loc[pl_id]['track_ids']:
                    correct += 1
                    ap += correct / (it+1)
            ap /= len(pred)
            mean_ap += ap
        
        counter += 1
        if counter % 1000 == 0:
            print(counter)
            if compute_MAP:
                print(mean_ap / counter)
    predictions['playlist_id'] = predictions['playlist_id_tmp']
    return predictions

In [366]:
#%%time
predictions = make_predictions(test=test, compute_MAP=True)

1000
0.07674666666666677
2000
0.07922166666666648
3000
0.07679333333333296


KeyboardInterrupt: 

<div style="white-space: pre-wrap;"><h4>single ones:</h4>
albums: 0.063
artists: 0.054
tags: 0.041
URM: 0.035 with k = 400
duration: 0.0002
playcount: 0.0004
playlist title similarity * URM not normalized: 0.004
<h4>albums + artists:</h4>
BETA = 0.5: 0.074
BETA = 0.75: 0.074
BETA = 0.9: 0.075
BETA = 1: 0.075

Chosen BETA: 0.9
<h4>albums + artists + tags:</h4>
GAMMA = 0.8: 0.076
GAMMA = 0.6: 
</div>

In [98]:
list(map(lambda l: sum(l)/len(l) if len(l)>0 else 0, IAM_album.data[:100]))

[11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356849563,
 11.107455356

# SVD supervised

In [ ]:
def from_num_to_id(df, row_num, column = 'track_id'):
    """ df must have a 'track_id' column """
    return df.iloc[row_num][column]

def from_id_to_num(df, tr_id, column='track_id'):
    """ df must have a 'track_id' column """
    return np.where(df[column].values == tr_id)[0][0]

In [ ]:
def build_id_to_num_map(df, column):
    a = pd.Series(np.arange(len(df)))
    a.index = df[column]
    return a

def build_num_to_id_map(df, column):
    a = pd.Series(df[column])
    a.index = np.arange(len(df))
    return a

In [ ]:
N_FEATURES = 5
N_EPOCHS = 5

In [ ]:
userValue = np.zeros((URM.shape[0], N_FEATURES))
userValue += 0.1

In [ ]:
itemValue = np.zeros((N_FEATURES,URM.shape[1]))
itemValue += 0.1

In [ ]:
def predictRating(user, item, features):
    return np.dot(userValue[user,:features+1], itemValue[:features+1,item])

In [ ]:
lrate = 0.01
K = 0.02
def train_user(user, item, rating, feature):
    err = (rating - predictRating(user, item, feature))
    
    userValue[user,feature] += lrate * (err * itemValue[feature,item] - K*userValue[user,feature])
    itemValue[feature,item] += lrate * (err * userValue[user,feature] - K*itemValue[feature, item])

In [ ]:
URM = URM.tocoo()

In [ ]:
%%time
for f in range(N_FEATURES):
    for i in range(N_EPOCHS):
        print("training feature {0}, stage {1}".format(f, i))
        for r,c in zip(URM.row, URM.col):
            train_user(r, c, 1, f)

In [ ]:
userValue

In [ ]:
itemValue

In [ ]:
sum((URM[r,c] - predictRating(r,c,N_FEATURES-1))**2 for r,c in zip(URM.row, URM.col))